# Recommendation System

## Задание 1

Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). Оставь в датасете только фильмы, которые вышли в "релиз".\
Выведи количество фильмов, оставшихся после фильтрации.

In [1]:
import pandas as pd

# Load data from files
movies_dataset = pd.read_csv('../datasets/tmdb_5000_movies.csv')
credits_dataset = pd.read_csv('../datasets/tmdb_5000_credits.csv')

# Merge data using the movie_id as the identifier
data = pd.merge(movies_dataset, credits_dataset, left_on='id', right_on='movie_id')

# Filter data based on the "status" column with value "Released"
dataset = data[data['status'] == 'Released'].copy()

# Remove duplicate columns
dataset.rename(columns={'title_x': 'title'}, inplace=True)
dataset.drop(columns='title_y', inplace=True)
dataset.drop(columns='movie_id', inplace=True)

num_movies = len(dataset)
print("количество фильмов вышедших в релиз:", num_movies)


количество фильмов вышедших в релиз: 4795


## Задание 2

Самый наивный подход к рекомендации фильмов - рекомендовать фильмы с лучшими оценками пользователей. Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$ 
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов 

Имплементируй функцию `weighted_rating`. С её помощью расcчитай рейтинг для каждого фильма и сохрани его в колонку `simple_score`.\
Выведи топ-5 фильмов по получившемуся рейтингу.
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [2]:
def weighted_rating(movie, num_votes_for_table, average_rating_all_movies):
    """
    Calculates the weighted rating for a movie using the IMDB weighted rating formula.

    Arguments:
        movie (pd.Series): Row of the movie data table.
        num_votes_for_table (float): Number of votes to include the movie in the final table (m).
        average_rating_all_movies (float): Average rating of all movies (C).

    Returns:
        float: Weighted rating for the movie.
    """
    num_votes = movie['vote_count']  # number of votes (v)
    average_rating = movie['vote_average']  # average rating (R)
    return (num_votes / (num_votes + num_votes_for_table) * average_rating) + (num_votes_for_table / (num_votes + num_votes_for_table) * average_rating_all_movies)


# Calculation of num_votes_for_table (m) and average_rating_all_movies (C)
num_votes_for_table = dataset['vote_count'].quantile(0.95)
average_rating_all_movies = dataset['vote_average'].mean()

# Calculation of the rating for each movie and saving it in the "simple_score" column
dataset['simple_score'] = dataset.apply(weighted_rating, args=(num_votes_for_table, average_rating_all_movies), axis=1)

# Getting the top 5 movies by rating
top_movies = dataset.nlargest(5, 'simple_score')

print("топ-5 фильмов по рейтингу (simple_score):")
print(top_movies[['title', 'simple_score']].to_string(index=False))


топ-5 фильмов по рейтингу (simple_score):
                   title  simple_score
The Shawshank Redemption      7.849025
         The Dark Knight      7.773990
              Fight Club      7.761012
               Inception      7.736496
            Pulp Fiction      7.714726


## Задание 3

Такой подход к рекомендациям очень наивен, так как не учитывает информацию о самом фильме (жанр, режиссер, описание, актеры и т.п). \
**Content Based Filtering** (Фильтрация на основе содержания) - тип рекомендательной системы, которая предлагает пользователям похожие элементы на основе конкретного элемента. Общая идея этих рекомендательных систем заключается в том, что если человеку понравился определенный товар, то ему понравится и похожий на него товар.

<center><img src="../misc/images/content.png" alt= “” width="300" height="500">

Реализуем алгоритм рекомендации на основе описания фильма. Для это требуется провести предобработку текста:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Расcчитай [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильма

Выведи размер получившейся матрицы Tf-Idf

> Для [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) используйте параметры по умолчанию 

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Replacing NaN in movie overview with an empty string
dataset['overview'].fillna('', inplace=True)

# Creating an instance of TfidfVectorizer with English stop words removal
vectorizer = TfidfVectorizer(stop_words='english')

# Calculation of Tf-Idf for movie overview
tfidf_matrix = vectorizer.fit_transform(dataset['overview'])

print("размер матрицы Tf-Idf:", tfidf_matrix.shape)


размер матрицы Tf-Idf: (4795, 20970)


## Задание 4

Теперь тебе необходимо вычислить показатель сходства между описаниями фильмов. Используем косинусное расстояние, оно рассчитывается по формуле:
$$cos(Θ) = \frac{A ⋅ B}{∥A∥ ∥B∥} = \frac{ Σ_{i=1}^{n} A_i ⋅ B_i } { \sqrt{Σ_{i=1}^{n}A_{i}^{2}} ⋅ {\sqrt{Σ_{i=1}^{n}B_{i}^{2}}}}$$
Но поскольку мы использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства. Рассчитать его можно через [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html). Результат сохрани в переменную `cosine_sim`.

Выведи размер получившейся матрицы `cosine_sim`.

In [4]:
from sklearn.metrics.pairwise import linear_kernel

# Calculation of similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print("размер матрицы `cosine_sim`:", cosine_sim.shape)


размер матрицы `cosine_sim`: (4795, 4795)


## Задание 5

Напиши функцию `get_recommendations`. На вход она принимает:
* `movies_dataset` - датасет фильмов
* `title` - название фильма, для которого мы будем искать похожие
* `cosine_sim` - матрица расстояний между описаниями
* `top_k` - топ-k cхожих фильмов

Возвращает top_k названий фильмов, описание которых похоже на выбранный фильм.\
Выведи топ-5 фильмов для `title='Saving Private Ryan'`

In [5]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):
    """
    Get recommendations of similar movies for a given movie.

    Args:
        movies_dataset (DataFrame): Dataset of movies.
        title (str): Title of the movie for which recommendations are needed.
        cosine_sim (ndarray): Matrix of distances between movie descriptions.
        top_k (int, optional): Number of top recommended movies. Default is 10.

    Returns:
        Series: Titles of the top-K recommended movies.

    Raises:
        IndexError: If the movie title is not found in the dataset.

    """
    try:
        # Create an index for fast access to movies by title
        indices = pd.Series(movies_dataset.index, index=movies_dataset['title']).drop_duplicates()
        idx = indices[title]
        # Get similarity scores for the given movie
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort movies by similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get indexes of top-K movies
        top_indexes = [score[0] for score in sim_scores[1:top_k+1]]

        # Get titles of top-K movies
        top_movies = movies_dataset['title'].iloc[top_indexes]

        return top_movies
    except KeyError:
        raise IndexError(f"Название фильма '{title}' не найдено в датасете.")


# Get recommendations for the movie 'Saving Private Ryan'
try:
    recommended_movies = get_recommendations(movies_dataset=dataset, title='Saving Private Ryan', cosine_sim=cosine_sim, top_k=5)
    print("топ-5 рекомендованных фильмов для 'Saving Private Ryan':")
    print(recommended_movies.to_string(index=False))
except IndexError as e:
    print(str(e))


топ-5 рекомендованных фильмов для 'Saving Private Ryan':
   The Great Raid
The Monuments Men
The Expendables 2
        Abandoned
        The Train


## Задание 6

Еще один подход к построению рекомендательной системы - подход на основе сходства между пользователями. Этот подход называется **Collaborative Filtering** (Коллаборативная фильтрация).
<center><img src="../misc/images/all.png" alt= “” width="600" height="700"></center>
Коллаборативная фильтрация - это тип рекомендательной системы, которая использует поведение и предпочтения похожих пользователей для рекомендации товаров или продуктов конкретному пользователю. Система собирает данные о прошлом поведении пользователей, такие как покупки, рейтинги и отзывы, и анализирует их для выявления закономерностей и сходства между пользователями. На основе этих закономерностей система рекомендует товары, которые понравились или были приобретены другими такими же пользователями в прошлом.

Для реализации Коллаборативной фильтрации нам потребуются оценки пользователей [ratings](../datasets/ratings.csv).

>userId - id пользователя \
movieId - id фильма \
rating - оценка фильма (от 0 до 5)\
timestamp - время оценки


Воспользуйся библиотекой [surprise](https://surpriselib.com/) для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD). Выведи средние значения 'RMSE', 'MAE' на кросс-валидации с параметрами `cv=5`.

In [6]:
#!pip install scikit-surprise

In [7]:
import numpy as np
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

# Load ratings data
ratings = pd.read_csv('../datasets/ratings.csv')

# Initialize Reader object
reader = Reader(rating_scale=(0, 5))

# Load data into Dataset object
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Initialize SVD model
model = SVD()

# Perform cross-validation with cv=5
cv_results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5)

print('средний RMSE на кросс-валидации:', np.mean(cv_results['test_rmse']))
print('средний MAE на кросс-валидации:', np.mean(cv_results['test_mae']))


средний RMSE на кросс-валидации: 0.8954165095312632
средний MAE на кросс-валидации: 0.6895805121725338
